# Post Call Analytics
Welcome to this training module on post-call analytics use cases using Amazon SageMaker JumpStart.

As businesses continue to interact with customers through various channels, it becomes increasingly important to analyze these interactions to gain insights into customer behavior and preferences. Post-call analytics is one such method that involves analyzing customer interactions after the call has ended. The use of large language models can greatly enhance the effectiveness of post-call analytics by enabling more accurate sentiment analysis, identifying specific customer needs and preferences, and improving overall customer experience.

In this sample notebook, we will explore following topics to demonstrate the various benefits of using Bedrock for post-call analytics and businesses gain a competitive edge in the modern marketplace.

* One model handling multiple PCA tasks <BR>
* Handling long call transcripts <BR>

## Step 0. Install packages

In [24]:
install_needed = True  # should only be True once

In [25]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U transformers
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.4 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.14.1 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.3 MB/s eta 0:00:00:00:010:01
  Obtaining depen

## Step 1. SageMaker Endpoint Wrapper

In [1]:
import json
import boto3
from pprint import pprint
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint

In [2]:
class VarcoContentHandler(LLMContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        입력 데이터 전처리 후에 리턴
        '''
        
        payload = {"text": prompt}
        payload.update(model_kwargs)
        input_str = json.dumps(payload)

        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> str:
    
        response_json = json.loads(output.read().decode("utf-8"))     
        generated_text = response_json["result"]        

        return generated_text[0]

In [3]:
aws_region = boto3.Session().region_name
llm_text_content_handler = VarcoContentHandler()
endpoint_name_text = "varco-llm-13b-ist"

In [4]:
params = {
    "repetition_penalty": 1.1,
    "temperature": 0.9,
    "top_k": 50,
    "top_p": 0.9
}

llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    model_kwargs=params,
    content_handler=llm_text_content_handler,
)

## Step 2. Load transcript files

In [5]:

transcript_files = [
    "./call_transcripts/negative-refund-ko.txt",
    "./call_transcripts/neutral-short-ko.txt",
    "./call_transcripts/positive-partial-refund-ko.txt",
    "./call_transcripts/aws-short-ko.txt",
]
transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())


for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

transcript #1: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 



transcript #2: timestamp: 2023-01-28 08:26:49.219717

고객: 안녕하세요, 제 계정의 잔액을 확인하고 싶습니다.

상담원: 물론이죠! 계정에 연결된 계좌 번호나 전화번호를 알려주실 수 있나요?

고객: 네: 제 전화번호는 (123) 456-7890입니다.

상담원: 네, 감사합니다. 계정을 불러올게요. 현재 잔액이 $567.89인 것 같습니다.

고객: 네, 좋아요. 감사합니다.

상담원: 천만에요! 오늘 또 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 감사합니다.

상담원: 문제 없습니다. 전화해 주셔서



transcript #3: timestamp: 2022-12-28 08:26:49.219717

상담원: 소매업체]에 전화해 주셔서 감사합니다. 제 이름은 [상담원 이름]입니다. 오늘은 무엇을 도와드릴까요?

고객: 안녕하세요, 주문 상태를 확인하고 싶어서요. 오늘 도착하기로 되어 있었는데 아직 받지 못했습니다.

상담원: 유감입니다. 주문 번호를 알려주시겠습니까?

고객: 네, 123456입니다.

상담원: 네: 감사합니다. 제가 확인해 보겠습니다. 창고에서 예기치 않은 문제가 발생하여 주문이 며칠 지연된 것 같습니다. 불편을 드려 죄송합니다.

고객: 괜



transcript #4: AWS란 무엇인가요? AWS 또는 Amazon We

## Step 3. Post Call Analysis

### Step 3.1. Prompt Template
In this notebook, we'll be performing four different analyses(Summary, Sentiment, Intent and Resolution), and we'll need a template for each one.

* Summary template

In [6]:
summary_template = """### User:
다음 대화를 간단하게 요약해 주세요.
대화: {transcript}


### Assistant:
"""

* Sentiment template

In [7]:
sentiment_template = """### User:
감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.
대화: {transcript}


### Assistant:
"""

* intent template

In [8]:
intent_template = """### User:
이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 
대화: {transcript}


### Assistant:
"""

### Step 3.2. Analysis

In [21]:
from termcolor import colored
from langchain import PromptTemplate

In [22]:
def analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    analysis_prompt = prompt.format(transcript=transcript)
    llm.model_kwargs = params

    print (colored(analysis_prompt, 'green'))

    response = llm(analysis_prompt)

    return response

In [37]:
params = {
    "repetition_penalty": 1.01,
    "temperature": 0.9,
    "top_k": 50,
    "top_p": 0.9
}

* summary

In [38]:
%%time

res = analysis(
    llm=llm_text,
    transcript=transcripts[0],
    params=params,
    template=summary_template
)

print (res)

### User:
다음 대화를 간단하게 요약해 주세요.
대화: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번 불편을 끼쳐 드려 죄송합니다. 소매 지원 라인에 문의해 주셔서 감사드리며 좋은 하루 되세요.


### Assistant:

### User:
### User:
다음 대화를 간단하게 요약해 주세요.
대화: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은

* sentiment

In [39]:
%%time

res = analysis(
    llm=llm_text,
    transcript=transcripts[0],
    params=params,
    template=sentiment_template
)

print (res)

### User:
감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.
대화: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번 불편을 끼쳐 드려 죄송합니다. 소매 지원 라인에 문의해 주셔서 감사드리며 좋은 하루 되세요.


### Assis

* intent

In [40]:
%%time

res = analysis(
    llm=llm_text,
    transcript=transcripts[0],
    params=params,
    template=intent_template
)

print (res)

### User:
이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 
대화: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번 불편을 끼쳐 드려 죄송합니다. 소매 지원 라인에 문의해 주셔서 감사

## Handling long call transcripts
We'll cover how to handle long transcripts that exceed the limits of the LLM.

In [41]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

* prompting to divide and conquer

In [42]:
stuff_prompt_template = """### User:
다음 글을 간단하게 요약해 주세요.
글: {text}

### Assistant:
"""

chuck_prompt_template = """### User:
다음 글을 간단하게 요약해 주세요.
글: {text}

### Assistant:
"""

chunk_prompt = PromptTemplate(
    template=chuck_prompt_template,
    input_variables=["text"]
)

combine_prompt_template = """### User:
다음 글을 간단하게 요약해 주세요.
글: {text}


### Assistant:
"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template,
    input_variables=["text"]
)

* summarize chain

In [43]:
'''
# summary_chain = load_summarize_chain(
#     llm=llm,
#     chain_type="map_reduce",
#     verbose=True
# ) # map_reduce, refine
# transcript = summary_chain(docs)
'''


def summary_chain_init(chain_type, llm):
    
    if chain_type == "STUFF":
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            verbose=True
        )
        
    elif chain_type == "MAP_REDUCE":
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=chunk_prompt,
            combine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
    elif chain_type == "REFINE":
        chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=chunk_prompt,
            refine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
        
    return chain

In [44]:
def long_call_analysis(llm, transcript, params, template="", chain_type="MAP_REDUCE", max_tokens=50):

    
    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning

    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

        
        summary_chain = summary_chain_init(
            chain_type=chain_type, 
            llm=llm
        )
        response = summary_chain(
            {"input_documents": docs}
        )
        
        print ("Intermediate_steps: \n")
        for idx, step in enumerate(response["intermediate_steps"]):
            print (colored(f'step {idx}: \n', "green"))
            print (colored(f'{step}\n', "green"))
        
        return response["output_text"]
    
    else:
        
        prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])
        analysis_prompt = prompt.format(text=transcript)
        print (colored(analysis_prompt, 'green'))
        
        response = llm(analysis_prompt)
        
        return response
        

In [45]:
params = {
    "repetition_penalty": 1.01,
    "temperature": 0.1,
    "top_k": 50,
    "top_p": 0.9
}

In [46]:

%%time

res = long_call_analysis(
    llm=llm_text,
    transcript=transcripts[3],
    params=params,
    template=summary_template,
    chain_type="REFINE" # REFINE, MAP_REDUCE
)

print ("Results: \n")
print (res)

Token indices sequence length is longer than the specified maximum sequence length for this model (2950 > 1024). Running this sequence through the model will result in indexing errors


Now we have 4 documents and the first one has 1001 tokens


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### User:
다음 글을 간단하게 요약해 주세요.
글: AWS란 무엇인가요? AWS 또는 Amazon Web Services는 공용 인터넷을 통해 액세스할 수 있는 다양한 컴퓨팅 서비스를 제공하는 클라우드 서비스 제공업체입니다.

AWS 및 기타 퍼블릭 클라우드 공급업체(예: Google Cloud Platform(GCP) 및 Microsoft Azure)는 하드웨어와 인프라를 관리 및 유지 관리하여 조직과 개인이 현장에서 리소스를 구매하고 실행하는 데 드는 비용과 복잡성을 덜어줍니다. 이러한 리소스는 무료 또는 사용량 기반 유료로 액세스할 수 있습니다.

AWS를 더 잘 이해하려면 AWS가 얼마나 방대한지 이해하는 것이 도움이 될 수 있습니다. 부정할 수 없는 사실은 AWS가 엄청나게 크다는 것입니다. 얼마나 큰 규모일까요?

인터넷에서 방문하는 사이트 세 곳 중 한 곳은 AWS 서비스를 사용합니다. 
2019년 아마존 웹 서비스는 350억 달러 이상의 매출을 올렸습니다. AWS가 단독 기업이었다면 포춘지 선정 글로벌 500대 기업에서 359위를 차지할 수 있는 규모입니다.
이제 AWS에 대한 10,000피트 높이의 개요를 살펴보겠습니다. 자세히 살펴보겠습니다!

### Assistant:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
### User:
다음 글을 간단하게 요약해 주세요.
글: AWS의 역사
AWS의 기원은 의도치 않게 시작되었습니다. 2000년 무렵만 해도 Amazon은 스타트업 시절부터 쌓인 기술 부채로 인해 성장에 어려움을 겪고 있던 허름한 이커머스